In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus

In [2]:
# Device configuration
device = torch.device('cpu')

In [3]:
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000     # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [4]:
# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data('../../data/language/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [5]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

In [8]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/3481], Loss: 10.4152, Perplexity: 33361.58
Epoch [1/5], Step[100/3481], Loss: 6.6405, Perplexity: 765.45
Epoch [1/5], Step[200/3481], Loss: 6.6232, Perplexity: 752.35
Epoch [1/5], Step[300/3481], Loss: 6.2822, Perplexity: 534.97
Epoch [1/5], Step[400/3481], Loss: 6.0880, Perplexity: 440.53
Epoch [1/5], Step[500/3481], Loss: 6.0540, Perplexity: 425.82
Epoch [1/5], Step[600/3481], Loss: 5.8358, Perplexity: 342.34
Epoch [1/5], Step[700/3481], Loss: 5.6816, Perplexity: 293.42
Epoch [1/5], Step[800/3481], Loss: 5.4420, Perplexity: 230.90
Epoch [1/5], Step[900/3481], Loss: 5.7951, Perplexity: 328.69
Epoch [1/5], Step[1000/3481], Loss: 5.7824, Perplexity: 324.54
Epoch [1/5], Step[1100/3481], Loss: 5.8132, Perplexity: 334.70
Epoch [1/5], Step[1200/3481], Loss: 5.6095, Perplexity: 273.01
Epoch [1/5], Step[1300/3481], Loss: 5.6688, Perplexity: 289.67
Epoch [1/5], Step[1400/3481], Loss: 5.6921, Perplexity: 296.52
Epoch [1/5], Step[1500/3481], Loss: 5.5183, Perplexity: 249.21
E

Epoch [4/5], Step[2800/3481], Loss: 3.1975, Perplexity: 24.47
Epoch [4/5], Step[2900/3481], Loss: 3.0659, Perplexity: 21.45
Epoch [4/5], Step[3000/3481], Loss: 3.2836, Perplexity: 26.67
Epoch [4/5], Step[3100/3481], Loss: 3.0131, Perplexity: 20.35
Epoch [4/5], Step[3200/3481], Loss: 2.8731, Perplexity: 17.69
Epoch [4/5], Step[3300/3481], Loss: 3.1699, Perplexity: 23.81
Epoch [4/5], Step[3400/3481], Loss: 3.4111, Perplexity: 30.30
Epoch [5/5], Step[0/3481], Loss: 3.2971, Perplexity: 27.04
Epoch [5/5], Step[100/3481], Loss: 3.2346, Perplexity: 25.40
Epoch [5/5], Step[200/3481], Loss: 3.3147, Perplexity: 27.51
Epoch [5/5], Step[300/3481], Loss: 3.1963, Perplexity: 24.44
Epoch [5/5], Step[400/3481], Loss: 3.1252, Perplexity: 22.76
Epoch [5/5], Step[500/3481], Loss: 3.0337, Perplexity: 20.77
Epoch [5/5], Step[600/3481], Loss: 2.8760, Perplexity: 17.74
Epoch [5/5], Step[700/3481], Loss: 3.0141, Perplexity: 20.37
Epoch [5/5], Step[800/3481], Loss: 3.0460, Perplexity: 21.03
Epoch [5/5], Step[9

In [9]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt
